# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [2]:
X = df.drop(['salary_in_usd'], axis = 1)
y = df['salary_in_usd']

In [3]:
X = X.drop(['salary'], axis = 1) #удаляем признак, из-за которого возможен лик в данных

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

columns_to_scale  = X_train.select_dtypes(np.number).columns
columns_to_encode = X_train.select_dtypes(include=['object']).columns

scaler = StandardScaler()
ohe = OneHotEncoder(sparse_output = False)

scaled_train  = scaler.fit_transform(X_train[columns_to_scale])
scaled_test  = scaler.transform(X_test[columns_to_scale])

encoded_train = ohe.fit_transform(X_train[columns_to_encode])
encoded_test = ohe.transform(X_test[columns_to_encode])

final_train = np.concatenate([scaled_train, encoded_train], axis = 1)
final_test = np.concatenate([scaled_test, encoded_test], axis=1)



In [6]:
model = LinearRegression()

model.fit(final_train, y_train)
y_pred = model.predict(final_test)
y_train_pred = model.predict(final_train)

In [7]:
print('MAPE: ', mean_absolute_percentage_error(y_test, y_pred))
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))

MAPE:  0.29349617570232334
RMSE:  45590.56479439181


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [10]:
from xgboost.sklearn import XGBRegressor

In [11]:
columns_to_scale  = X_val.select_dtypes(np.number).columns
columns_to_encode = X_val.select_dtypes(include=['object']).columns

scaler = StandardScaler()
ohe = OneHotEncoder(sparse_output = False)

scaled_val  = scaler.fit_transform(X_val[columns_to_scale])

encoded_val = ohe.fit_transform(X_val[columns_to_encode])

final_val = np.concatenate([scaled_val, encoded_val], axis = 1)

In [12]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np


params = {
    'max_depth' : [5, 7, 9, 12, 15],
    'learning_rate' : np.arange(0.01,0.2,0.005),
    'n_estimators' : np.arange(1, 50, 5),
    'gamma' : np.arange(0, 1, 0.1)

}

model = XGBRegressor()

cv = [(slice(None), slice(None))]

grid_search = RandomizedSearchCV(model, params, scoring='neg_root_mean_squared_error', n_iter = 30, cv = cv)
grid_result = grid_search.fit(final_val,y_val)

In [13]:
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'n_estimators': 46, 'max_depth': 15, 'learning_rate': 0.18, 'gamma': 0.0}


In [14]:
model = XGBRegressor(n_estimators = 46, max_depth = 15, learning_rate = 0.18, gamma = 0.0)

model.fit(final_train, y_train)
y_pred = model.predict(final_test)
y_train_pred = model.predict(final_train)

print('MAPE: ', mean_absolute_percentage_error(y_test, y_pred))
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))

MAPE:  0.23140465010646127
RMSE:  40224.15555899208


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [16]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.8 MB/s eta 0:00:00


In [17]:
from catboost import CatBoostRegressor

params = {
    'depth' : [5, 7, 9, 12, 15],
    'learning_rate' : np.arange(0.01,0.2,0.005),
    'iterations' :  [300, 400, 500],

}

model = CatBoostRegressor(verbose = False)

cv = [(slice(None), slice(None))]

#kfold = KFold(n_splits=10, shuffle=True, random_state=42)
grid_search = RandomizedSearchCV(model, params, scoring='neg_root_mean_squared_error', n_iter = 30, cv = cv)
grid_result = grid_search.fit(final_val,y_val)

In [18]:
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'learning_rate': 0.12999999999999998, 'iterations': 500, 'depth': 15}


In [19]:
model = CatBoostRegressor(learning_rate = 0.13, iterations = 500, depth = 15, verbose = False)

model.fit(final_train, y_train)
y_pred = model.predict(final_test)
y_train_pred = model.predict(final_train)

print('MAPE: ', mean_absolute_percentage_error(y_test, y_pred))
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))

MAPE:  0.21822351248615787
RMSE:  39944.335072719994


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [25]:
from catboost import Pool, cv


pool = Pool(X_train, y_train, cat_features = list(columns_to_encode))

params = {
    'depth' : [5, 7, 9, 12, 15],
    'learning_rate' : np.arange(0.01,0.2,0.005),
    'iterations' :  [300, 400, 500],

}

model = CatBoostRegressor(verbose = False)

#cv = [(slice(None), slice(None))]



grid_search = model.randomized_search(params, pool, n_iter = 20, verbose = False, search_by_train_test_split = False, calc_cv_statistics = False)


Training on fold [0/3]

bestTest = 46289.98267
bestIteration = 382

Training on fold [1/3]

bestTest = 48538.38053
bestIteration = 397

Training on fold [2/3]

bestTest = 49670.49258
bestIteration = 399

Training on fold [0/3]

bestTest = 46256.99609
bestIteration = 163

Training on fold [1/3]

bestTest = 48573.66938
bestIteration = 265

Training on fold [2/3]

bestTest = 49623.02065
bestIteration = 398

Training on fold [0/3]

bestTest = 46465.76569
bestIteration = 87

Training on fold [1/3]

bestTest = 48779.26309
bestIteration = 184

Training on fold [2/3]

bestTest = 49125.35213
bestIteration = 207

Training on fold [0/3]

bestTest = 46129.36884
bestIteration = 115

Training on fold [1/3]

bestTest = 48193.11641
bestIteration = 276

Training on fold [2/3]

bestTest = 49488.49815
bestIteration = 453

Training on fold [0/3]

bestTest = 46383.54775
bestIteration = 172

Training on fold [1/3]

bestTest = 48299.6678
bestIteration = 137

Training on fold [2/3]

bestTest = 49583.23589
bes

KeyboardInterrupt: 

In [21]:
print(grid_search)

{'params': {'depth': 12, 'learning_rate': 0.28850000000000003, 'iterations': 9}}


In [ ]:
model = CatBoostRegressor(learning_rate = 0.28850000000000003, iterations = 9, depth = 12, verbose = False)

model.fit(final_train, y_train)
y_pred = model.predict(final_test)
y_train_pred = model.predict(final_train)

print('MAPE: ', mean_absolute_percentage_error(y_test, y_pred))
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))

MAPE:  0.3738283073435365
RMSE:  47556.92689417931


**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [38]:
from lightgbm import LGBMRegressor


params = {
    'max_depth' : [25, 50, 75],
    'learning_rate' : [0.01,0.05,0.1],
    'n_estimators' : [300, 400, 500],
    "num_leaves": [300,900,1200]
}

model = LGBMRegressor(verbose = -1)

cv = [(slice(None), slice(None))]


grid_search = RandomizedSearchCV(model, params, scoring='neg_root_mean_squared_error', n_iter = 30, cv = cv)
grid_result = grid_search.fit(final_val,y_val)

In [39]:
print("Best Hyperparameters:", grid_search.best_params_)

Best Hyperparameters: {'num_leaves': 300, 'n_estimators': 500, 'max_depth': 25, 'learning_rate': 0.1}


In [40]:
model = LGBMRegressor(n_estimators = 500, max_depth = 25, learning_rate = 0.1, num_leaves = 300, verbose = -1)

model.fit(final_train, y_train)
y_pred = model.predict(final_test)
y_train_pred = model.predict(final_train)

print('MAPE: ', mean_absolute_percentage_error(y_test, y_pred))
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))

MAPE:  0.26686299920145234
RMSE:  42525.521322811655


In [34]:
model

LGBMRegressor(verbose=-1)

## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

**Ответ:** # -- YOUR ANSWER HERE --

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [ ]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [ ]:
# -- YOUR CODE HERE --

Выкиньте строку под названием `user`.

In [ ]:
# -- YOUR CODE HERE --

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [ ]:
# -- YOUR CODE HERE --
ratings.sample()

## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [ ]:
from sklearn.cluster import KMeans

# -- YOUR CODE HERE --

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --